In [6]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from extract import extract_covid_data
from transformation import transform_data
from load import load_data
from datetime import datetime
import argparse
from pyspark.sql.functions import map_concat, lit, create_map, explode, col
from pyspark.sql.types import DoubleType

In [7]:
def get_spark_utils():
    """
    !!!DO NOT TOUCH!!!
    This function returns spark context object and spark session object.
    These are the entry point into all functionality in Spark.
    :return: SparkContext, SparkSession
    """
    conf = SparkConf().setAppName("Covid"). \
        set("spark.mongodb.input.uri", "mongodb://127.0.0.1"). \
        set("spark.mongodb.output.uri", "mongodb://127.0.0.1"). \
        set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1"). \
        set("spark.sql.debug.maxToStringFields", 1000)
    spark = SparkSession.builder.master("local[*]").config(conf=conf).getOrCreate()
    sc = spark.sparkContext
    return sc, spark

In [8]:
sc, spark = get_spark_utils()

In [9]:
raw_data = extract_covid_data()

In [10]:
sdf = spark.createDataFrame(raw_data)

In [31]:
sdf.printSchema()

root
 |-- aged_65_older: double (nullable = true)
 |-- aged_70_older: double (nullable = true)
 |-- cardiovasc_death_rate: double (nullable = true)
 |-- continent: string (nullable = true)
 |-- data: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- diabetes_prevalence: double (nullable = true)
 |-- gdp_per_capita: double (nullable = true)
 |-- handwashing_facilities: double (nullable = true)
 |-- hospital_beds_per_thousand: double (nullable = true)
 |-- human_development_index: double (nullable = true)
 |-- life_expectancy: double (nullable = true)
 |-- location: string (nullable = true)
 |-- median_age: double (nullable = true)
 |-- population: double (nullable = true)
 |-- population_density: double (nullable = true)
 |-- extreme_poverty: double (nullable = true)
 |-- female_smokers: double (nullable = true)
 |-- male_smokers: double (nullable = true)



In [32]:
sdfe = sdf.select(sdf.location, explode(sdf.data))

In [33]:
sdfe.show(10)

22/12/26 19:42:54 WARN TaskSetManager: Stage 6 contains a task of very large size (17954 KiB). The maximum recommended task size is 1000 KiB.


+-----------+--------------------+
|   location|                 col|
+-----------+--------------------+
|Afghanistan|{date -> 2020-02-...|
|Afghanistan|{date -> 2020-02-...|
|Afghanistan|{date -> 2020-02-...|
|Afghanistan|{date -> 2020-02-...|
|Afghanistan|{date -> 2020-02-...|
|Afghanistan|{date -> 2020-02-...|
|Afghanistan|{date -> 2020-03-...|
|Afghanistan|{date -> 2020-03-...|
|Afghanistan|{date -> 2020-03-...|
|Afghanistan|{date -> 2020-03-...|
+-----------+--------------------+
only showing top 10 rows



In [36]:
sdfr = sdfe.rdd.map(lambda x: (x.location, x['col']['date'], x['col']['total_cases'], x['col']['new_cases'], x['col']['total_cases_per_million'], x['col']['new_cases_per_million']))
sdfre = sdfr.toDF(['location', 'date', 'total_cases', 'new_cases', 'total_cases_per_million', 'new_cases_per_million'])

22/12/26 19:43:27 WARN TaskSetManager: Stage 8 contains a task of very large size (17954 KiB). The maximum recommended task size is 1000 KiB.


In [37]:
sdfre.na.fill("")

DataFrame[location: string, date: string, total_cases: string, new_cases: string, total_cases_per_million: string, new_cases_per_million: string]

In [38]:
sdfre = sdfre.withColumn("total_cases", sdfre["total_cases"].cast(DoubleType()))
sdfre = sdfre.withColumn("new_cases", sdfre["new_cases"].cast(DoubleType()))
sdfre = sdfre.withColumn("total_cases_per_million", sdfre["total_cases_per_million"].cast(DoubleType()))
sdfre = sdfre.withColumn("new_cases_per_million", sdfre["new_cases_per_million"].cast(DoubleType()))

In [39]:
sdfre.printSchema()

root
 |-- location: string (nullable = true)
 |-- date: string (nullable = true)
 |-- total_cases: double (nullable = true)
 |-- new_cases: double (nullable = true)
 |-- total_cases_per_million: double (nullable = true)
 |-- new_cases_per_million: double (nullable = true)



In [41]:
sdfre.where(sdfre.date=='2021-05-03').show(10)

22/12/26 19:43:56 WARN TaskSetManager: Stage 10 contains a task of very large size (17954 KiB). The maximum recommended task size is 1000 KiB.
22/12/26 19:43:56 ERROR Executor: Exception in task 0.0 in stage 10.0 (TID 10)1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/var/folders/sc/88tgwthn4hx6v4hmbbwzp0fh0000gp/T/ipykernel_86304/1

Py4JJavaError: An error occurred while calling o733.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10) (161.9.123.126 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/var/folders/sc/88tgwthn4hx6v

22/12/26 20:19:51 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 421552 ms exceeds timeout 120000 ms
22/12/26 20:19:51 WARN SparkContext: Killing executors is not supported by current scheduler.
